In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('mol2vec.csv')
df.replace(to_replace='#NAME?',value=np.nan,inplace=True)
df.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = df.columns
df = df.astype("float", errors = "ignore")
df = np.float32(df)
df[np.isnan(df)] = 0
df[np.isinf(df)] = 0
df=pd.DataFrame(df,columns=columns)
ncol = df.shape[1]-1
x = df.iloc[:,0:ncol]
y = df['active']
corr_matrix = x.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
x1 = x.drop(to_drop, axis=1)
x2 = x1.loc[:, (x1 != x1.iloc[0]).any()] 
x2 = x2.round(6)
import warnings
warnings.filterwarnings("ignore")

In [2]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
0,-3.042170,-3.151222,-3.232676,7.560959,5.113686,-0.568981,-19.256578,-4.969994,16.522644,1.793195,...,12.655949,20.450172,4.086627,-10.276544,-2.190987,-1.115333,-9.388687,-15.132810,-4.057350,1.0
1,-4.727887,-4.351881,-4.658237,-1.792979,-0.499149,-8.724982,-8.886705,5.068110,-1.706909,-1.742328,...,14.233682,-0.425539,-0.464745,-10.994161,-12.039774,-12.723747,-4.175037,-11.076891,2.639213,1.0
2,-4.097823,-3.983348,-5.248144,-0.722321,-1.265023,-8.232691,-7.873246,5.745502,-0.900785,-1.008448,...,13.890897,-0.387179,0.128263,-11.509965,-10.279533,-11.952808,-3.907961,-10.316779,2.601157,1.0
3,-5.616269,-6.542370,-6.168895,-3.166161,1.509026,-11.890589,-10.365218,4.797748,-1.633804,-2.156420,...,16.483412,1.412684,-0.536551,-12.108947,-14.011013,-14.057915,-5.047316,-14.451323,1.707880,1.0
4,-4.396163,-5.507332,-5.355181,-2.798300,1.738860,-10.982598,-10.979731,4.270193,-1.071588,-2.923557,...,16.781738,-0.069494,-0.499126,-10.858052,-13.415128,-14.032432,-5.018421,-12.879906,1.733685,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,-0.245050,0.000723,-1.600506,2.734911,1.502080,0.887623,-5.741633,0.586250,4.324068,-0.374940,...,4.085058,5.202066,2.300924,-5.023169,-0.517164,-1.643741,-2.033808,-4.831588,0.884858,0.0
2020,-1.848185,-1.777678,-1.303566,0.651581,1.617647,1.635964,-2.599049,-0.967981,0.539179,1.095203,...,1.162264,3.851595,-0.020075,-1.346415,-1.949291,-0.575638,0.865116,-2.355083,-1.106393,0.0
2021,0.009402,0.458835,2.047133,-0.661602,1.160643,-0.072311,-4.937521,-2.287562,4.538611,-0.010992,...,1.278592,3.704251,0.990562,-2.375479,0.759892,-2.489954,-2.155487,-2.489589,-1.650008,0.0
2022,-3.098453,-1.974883,-2.323134,-0.688243,8.347803,0.660212,-16.231108,-3.275765,4.571203,-5.276364,...,3.041915,10.691944,3.452311,-7.664165,-4.948164,-1.304520,-8.855995,-8.327632,-5.991015,0.0


In [3]:
from sklearn.model_selection import train_test_split
x_train,x_val_test,y_train,y_val_test = train_test_split(x2,y,test_size = 0.2,random_state = 3)
x_val, x_test, y_val, y_test = train_test_split(x_val_test,y_val_test,test_size = 0.5,random_state = 3)

In [4]:
train = pd.concat([x_train,y_train],axis=1)
train

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
1600,-0.228303,-0.420543,-1.151437,2.137384,0.948651,-0.966793,-5.378348,-0.578722,4.473584,1.888198,...,4.518830,5.025622,0.864321,-4.249992,-0.857645,-2.313204,-1.992894,-5.150585,-1.131494,0.0
979,-0.924738,-3.630971,-0.941683,2.288410,0.979304,3.368458,-6.296511,-1.376926,0.691666,-4.330696,...,8.206991,2.423658,-2.514117,-6.368415,-6.289406,-3.261788,-4.375273,-5.743221,-2.720430,1.0
1671,-0.128579,-1.196362,-0.901492,-0.195001,0.187658,1.932691,-1.968215,0.821564,-0.328823,1.418066,...,3.314507,3.419686,1.748383,-1.418362,-0.482427,-1.750672,-1.698346,-1.072560,-1.650289,0.0
216,4.309278,-8.431256,-2.322263,7.220636,0.170849,-0.106744,-17.714476,-0.697247,8.181168,3.852382,...,17.693933,11.840370,-2.773586,-12.397419,-6.921757,-10.756232,-8.506141,-16.335634,-3.110963,1.0
1692,-0.987479,-0.858284,-1.361251,3.998539,-0.754172,1.200227,-8.802352,1.854344,6.913468,4.778369,...,7.425052,7.591580,-0.722258,-2.586851,0.752506,-1.687713,-0.424009,-3.616200,-0.364725,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,0.392188,1.968473,-0.943038,1.877094,1.254864,0.500375,-6.223106,0.767170,3.638936,2.147910,...,3.201548,5.102993,0.376566,-3.625142,-1.843430,-2.872926,-2.285060,-6.415958,-0.974725,1.0
1667,3.094696,-3.139788,0.452934,4.033810,3.162231,2.584005,-7.018096,-2.085129,4.469300,-2.197375,...,1.498193,5.878186,0.454574,-4.276176,-0.838577,-3.075898,-2.482870,-6.039039,-1.432760,0.0
1273,0.734135,-1.460896,0.043825,-0.108805,2.879373,1.196702,-3.448336,-0.502137,0.316171,-1.373874,...,0.880578,2.150828,-0.000220,-4.052219,-1.854702,-0.721018,-3.012404,-2.372019,-2.435756,0.0
1688,1.321631,-2.809979,-1.794365,2.394511,-0.813768,-2.125990,-6.982780,-0.949291,2.961360,2.448634,...,6.871832,4.953020,-0.890527,-4.268434,-3.141819,-3.867970,-2.545010,-5.462207,0.594360,0.0


In [5]:
val = pd.concat([x_val,y_val],axis=1)
val

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
768,1.930138,-4.962776,-3.553692,6.409678,-0.287666,-2.827934,-10.383595,1.374847,5.181082,2.954690,...,14.348227,6.225714,-1.330657,-10.411627,-6.792150,-7.941870,-4.480544,-12.984406,-0.617696,0.0
1493,-0.139839,-1.305578,-0.743600,1.292056,0.036296,-0.412776,-1.512009,-0.522574,1.453460,-1.086392,...,1.969465,3.412046,-0.949136,-0.653013,0.236782,-0.971184,-1.171660,-2.201481,-1.369430,0.0
373,4.143778,-11.520093,-10.596652,20.577940,3.682675,-20.086000,-38.062164,-4.725483,26.686539,-16.379730,...,37.040710,6.109267,6.991266,-38.453468,-35.007889,-20.426003,-22.350227,-53.638824,-5.012210,1.0
2015,-0.568558,-1.622268,-2.330306,6.046390,-0.417426,-0.045855,-8.051148,-1.140411,7.366740,3.022623,...,7.488421,7.709618,0.904696,-3.893689,-1.090966,-1.185946,-2.580934,-5.885628,-1.707931,0.0
631,0.967523,-0.911585,-1.082692,6.103718,-2.787804,-1.471303,-7.723426,2.328886,4.799963,2.295898,...,8.468033,5.037877,0.038659,-6.109335,-4.633872,-3.431532,-3.018868,-7.758264,1.976452,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,5.674945,-8.282511,-2.838195,11.207559,0.125618,2.714812,-15.374886,-0.066639,8.448422,3.163009,...,12.870675,18.011600,-1.302023,-9.646257,-1.905911,-8.111694,-4.823266,-13.279407,-2.860494,1.0
752,3.457904,-4.011414,-5.353265,10.390726,-1.322770,1.976202,-9.022825,-0.494663,9.446673,3.671260,...,10.045172,13.250098,-2.200596,-6.663626,-1.924840,-3.247668,-3.735476,-13.154430,-0.716472,1.0
2023,-2.087836,-0.969651,-2.509548,0.854120,3.467041,-2.948071,-6.558218,1.434711,1.283392,-0.060524,...,5.151205,1.269592,-0.919778,-6.967348,-5.402448,-4.868966,-1.531266,-7.708378,-2.107920,0.0
1888,0.123438,-0.320020,0.596228,-1.901501,4.888968,2.167732,-4.310668,1.116782,2.807673,-3.885461,...,2.767149,5.674098,2.106758,-5.057951,-1.908033,-1.790244,-5.421206,-6.590776,-3.989113,0.0


In [25]:
test1 = pd.concat([x_test,y_test],axis=1)
test1

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
1518,2.596721,-1.884308,-6.416350,5.696806,0.383852,-0.324962,-11.766581,1.148548,10.128831,6.070408,...,11.513759,11.571949,0.735413,-9.342612,-0.992301,-5.483562,-3.080058,-12.105997,-1.415661,0.0
288,1.542621,-4.220248,-4.080070,7.023908,2.067261,1.845866,-8.596121,-0.623660,3.652696,-1.755152,...,9.153687,10.263209,-1.468621,-5.136092,-3.716178,-4.128716,-3.374544,-9.262395,-2.355891,1.0
1915,-1.515145,-4.795324,-3.774084,0.506250,-1.169404,-7.843677,-8.642588,4.632508,0.644076,-2.062667,...,14.782564,-2.389532,-1.377842,-10.470648,-8.653366,-11.358960,-1.531673,-10.185263,2.609531,0.0
456,-0.117998,-0.185049,-0.241695,-0.934444,-3.578366,-1.048317,-3.021850,5.403191,4.046879,0.344293,...,6.918404,1.113004,-0.800505,-3.478112,-2.294981,-2.284974,-2.658378,-2.795206,-0.250958,1.0
751,1.326808,-1.295153,-6.008441,6.672756,2.358148,0.911839,-12.817696,-1.413208,5.093195,6.599452,...,6.793866,10.068006,-2.187404,-8.836248,-6.178484,-5.632662,-2.498834,-13.793689,-3.045668,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,0.226998,-0.568124,-0.160383,-0.321021,0.570335,0.870151,-3.193452,1.427354,-0.260336,1.253956,...,2.120289,1.896063,-0.749112,-2.455053,-0.965882,0.500872,-2.582612,-1.570571,-0.337280,0.0
1290,0.327324,-1.691828,-0.732084,0.011534,-0.031144,-0.000822,-1.439559,1.003249,3.093706,1.775479,...,1.383003,0.912382,-1.535159,-2.111576,-1.527270,-2.038852,-0.501496,-3.278867,-1.334571,0.0
896,6.449346,-7.739508,-5.712228,10.916770,-2.323613,3.474744,-14.996200,1.234536,6.079174,3.352927,...,18.199436,12.651924,-3.874452,-13.845347,-6.947412,-11.267908,-6.454752,-10.543712,-1.758062,0.0
2012,0.112171,-1.060670,-2.099404,6.525712,-0.744123,0.145420,-6.157840,0.190532,4.865452,2.658176,...,6.647238,6.427322,-0.127530,-3.216801,-0.926009,-2.814141,-2.013107,-4.369110,-0.685484,0.0


In [7]:
col_name = train.columns.values.tolist()
col_name.pop()

'active'

In [8]:
from pycaret.classification import *
CM = setup(data = train, target = 'active', test_data = val, silent=True, numeric_features = col_name,
             session_id=0, fold_strategy = 'stratifiedkfold', fold = 5, fold_shuffle = True, normalize = True, normalize_method = 'minmax')

,Description,Value
0,session_id,0
1,Target,active
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(1619, 300)"
5,Missing Values,False
6,Numeric Features,299
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [9]:
mlp = create_model('mlp')
tunedmlp = tune_model(mlp,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedmlp)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8302,0.8975,0.8472,0.7871,0.8161,0.6589,0.6604
1,0.8210,0.8744,0.7778,0.8116,0.7943,0.6360,0.6364
2,0.8519,0.9175,0.8611,0.8158,0.8378,0.7017,0.7025
3,0.8241,0.9026,0.8621,0.7716,0.8143,0.6481,0.6517
4,0.8390,0.8885,0.7986,0.8333,0.8156,0.6729,0.6733
Mean,0.8332,0.8961,0.8294,0.8039,0.8156,0.6635,0.6649
Std,0.0112,0.0143,0.0347,0.0219,0.0138,0.0226,0.0223


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,MLP Classifier,0.8317,0.8759,0.7955,0.8140,0.8046,0.6568,0.6569


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.670764,0.761575,0.712024,0.517488,0.180527,0.646217,0.823427,0.818446,0.259414,0.692151,...,0.146838,0.755971,0.803129,0.772490,0.848375,0.833408,0.501874,0.0,1.0,0.8693
1,0.599149,0.854475,0.804349,0.343203,0.190066,0.702282,0.934367,0.767116,0.221655,0.552314,...,0.157692,0.951564,0.937230,0.918342,0.892617,0.961623,0.472582,0.0,0.0,0.8829
2,0.747348,0.595007,0.480630,1.000000,0.297431,0.245596,0.477305,0.653418,0.477254,0.023106,...,0.383595,0.193926,0.264816,0.511278,0.609447,0.350004,0.330637,1.0,1.0,0.8705
3,0.584317,0.846431,0.752218,0.505116,0.176706,0.710799,0.852595,0.750403,0.281554,0.694501,...,0.210433,0.886611,0.911899,0.913848,0.873774,0.917817,0.459392,0.0,0.0,0.7618
4,0.637460,0.864483,0.793208,0.507068,0.106912,0.677710,0.856693,0.844255,0.255554,0.669354,...,0.185795,0.842202,0.844306,0.866863,0.867919,0.895550,0.602958,1.0,0.0,0.5672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.800321,0.677248,0.735532,0.680884,0.192696,0.774884,0.761011,0.779451,0.292511,0.699359,...,0.147652,0.771311,0.896351,0.768937,0.843793,0.829900,0.414481,1.0,1.0,0.7121
198,0.723619,0.785742,0.652900,0.653066,0.150049,0.757739,0.840444,0.767871,0.302622,0.716947,...,0.122088,0.831093,0.895990,0.870710,0.858337,0.831387,0.498025,1.0,1.0,0.8800
199,0.531755,0.863008,0.746329,0.328289,0.291082,0.643429,0.871264,0.820066,0.219932,0.587813,...,0.158527,0.825005,0.829642,0.836786,0.887809,0.896143,0.443806,0.0,0.0,0.6157
200,0.608258,0.879510,0.848368,0.234444,0.332950,0.762185,0.899370,0.811465,0.235373,0.455455,...,0.244632,0.863275,0.896311,0.901204,0.835798,0.909432,0.370503,0.0,0.0,0.7868


In [10]:
xgb = create_model('xgboost')
tunedxgb = tune_model(xgb,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedxgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8426,0.9288,0.9097,0.7751,0.8371,0.6867,0.6949
1,0.8364,0.9142,0.8681,0.7862,0.8251,0.6722,0.6751
2,0.8272,0.9198,0.8750,0.7683,0.8182,0.6548,0.6598
3,0.8241,0.9154,0.9379,0.7391,0.8267,0.6531,0.6723
4,0.8390,0.9108,0.9167,0.7674,0.8354,0.6803,0.6906
Mean,0.8339,0.9178,0.9015,0.7672,0.8285,0.6694,0.6785
Std,0.0071,0.0062,0.0263,0.0156,0.0070,0.0134,0.0128


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8416,0.9048,0.8750,0.7857,0.8280,0.6820,0.6853


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.670764,0.761575,0.712024,0.517488,0.180527,0.646217,0.823427,0.818446,0.259414,0.692151,...,0.146838,0.755971,0.803129,0.772490,0.848375,0.833408,0.501874,0.0,1.0,0.9864
1,0.599149,0.854475,0.804349,0.343203,0.190066,0.702282,0.934367,0.767116,0.221655,0.552314,...,0.157692,0.951564,0.937230,0.918342,0.892617,0.961623,0.472582,0.0,0.0,0.9844
2,0.747348,0.595007,0.480630,1.000000,0.297431,0.245596,0.477305,0.653418,0.477254,0.023106,...,0.383595,0.193926,0.264816,0.511278,0.609447,0.350004,0.330637,1.0,1.0,0.9982
3,0.584317,0.846431,0.752218,0.505116,0.176706,0.710799,0.852595,0.750403,0.281554,0.694501,...,0.210433,0.886611,0.911899,0.913848,0.873774,0.917817,0.459392,0.0,0.0,0.9703
4,0.637460,0.864483,0.793208,0.507068,0.106912,0.677710,0.856693,0.844255,0.255554,0.669354,...,0.185795,0.842202,0.844306,0.866863,0.867919,0.895550,0.602958,1.0,1.0,0.6342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.800321,0.677248,0.735532,0.680884,0.192696,0.774884,0.761011,0.779451,0.292511,0.699359,...,0.147652,0.771311,0.896351,0.768937,0.843793,0.829900,0.414481,1.0,1.0,0.9975
198,0.723619,0.785742,0.652900,0.653066,0.150049,0.757739,0.840444,0.767871,0.302622,0.716947,...,0.122088,0.831093,0.895990,0.870710,0.858337,0.831387,0.498025,1.0,1.0,0.9989
199,0.531755,0.863008,0.746329,0.328289,0.291082,0.643429,0.871264,0.820066,0.219932,0.587813,...,0.158527,0.825005,0.829642,0.836786,0.887809,0.896143,0.443806,0.0,0.0,0.9849
200,0.608258,0.879510,0.848368,0.234444,0.332950,0.762185,0.899370,0.811465,0.235373,0.455455,...,0.244632,0.863275,0.896311,0.901204,0.835798,0.909432,0.370503,0.0,0.0,0.9724


In [11]:
rf = create_model('rf')
tunedrf = tune_model(rf, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedrf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8519,0.9278,0.8819,0.8038,0.8411,0.7029,0.7055
1,0.8333,0.9127,0.7917,0.8261,0.8085,0.6611,0.6616
2,0.8302,0.9125,0.8125,0.8069,0.8097,0.6565,0.6565
3,0.8426,0.9157,0.8897,0.7866,0.8350,0.6856,0.6903
4,0.8452,0.9111,0.8403,0.8176,0.8288,0.6876,0.6878
Mean,0.8406,0.9160,0.8432,0.8082,0.8246,0.6787,0.6803
Std,0.0079,0.0061,0.0381,0.0134,0.0132,0.0174,0.0185


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8564,0.9107,0.8182,0.8471,0.8324,0.7069,0.7072


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.670764,0.761575,0.712024,0.517488,0.180527,0.646217,0.823427,0.818446,0.259414,0.692151,...,0.146838,0.755971,0.803129,0.772490,0.848375,0.833408,0.501874,0.0,1.0,0.7538
1,0.599149,0.854475,0.804349,0.343203,0.190066,0.702282,0.934367,0.767116,0.221655,0.552314,...,0.157692,0.951564,0.937230,0.918342,0.892617,0.961623,0.472582,0.0,0.0,0.9541
2,0.747348,0.595007,0.480630,1.000000,0.297431,0.245596,0.477305,0.653418,0.477254,0.023106,...,0.383595,0.193926,0.264816,0.511278,0.609447,0.350004,0.330637,1.0,1.0,0.8697
3,0.584317,0.846431,0.752218,0.505116,0.176706,0.710799,0.852595,0.750403,0.281554,0.694501,...,0.210433,0.886611,0.911899,0.913848,0.873774,0.917817,0.459392,0.0,0.0,0.8728
4,0.637460,0.864483,0.793208,0.507068,0.106912,0.677710,0.856693,0.844255,0.255554,0.669354,...,0.185795,0.842202,0.844306,0.866863,0.867919,0.895550,0.602958,1.0,0.0,0.6383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.800321,0.677248,0.735532,0.680884,0.192696,0.774884,0.761011,0.779451,0.292511,0.699359,...,0.147652,0.771311,0.896351,0.768937,0.843793,0.829900,0.414481,1.0,1.0,0.8609
198,0.723619,0.785742,0.652900,0.653066,0.150049,0.757739,0.840444,0.767871,0.302622,0.716947,...,0.122088,0.831093,0.895990,0.870710,0.858337,0.831387,0.498025,1.0,1.0,0.8957
199,0.531755,0.863008,0.746329,0.328289,0.291082,0.643429,0.871264,0.820066,0.219932,0.587813,...,0.158527,0.825005,0.829642,0.836786,0.887809,0.896143,0.443806,0.0,0.0,0.7844
200,0.608258,0.879510,0.848368,0.234444,0.332950,0.762185,0.899370,0.811465,0.235373,0.455455,...,0.244632,0.863275,0.896311,0.901204,0.835798,0.909432,0.370503,0.0,0.0,0.8321


In [12]:
et = create_model('et')
tunedet = tune_model(et, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedet)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8580,0.9272,0.8889,0.8101,0.8477,0.7153,0.7180
1,0.8333,0.9162,0.7847,0.8309,0.8071,0.6606,0.6615
2,0.8426,0.9165,0.8125,0.8298,0.8211,0.6806,0.6807
3,0.8457,0.9169,0.8966,0.7879,0.8387,0.6920,0.6973
4,0.8328,0.9164,0.8194,0.8082,0.8138,0.6621,0.6622
Mean,0.8425,0.9186,0.8404,0.8134,0.8257,0.6821,0.6839
Std,0.0093,0.0043,0.0443,0.0159,0.0152,0.0203,0.0216


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8515,0.9139,0.8182,0.8372,0.8276,0.6972,0.6973


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.670764,0.761575,0.712024,0.517488,0.180527,0.646217,0.823427,0.818446,0.259414,0.692151,...,0.146838,0.755971,0.803129,0.772490,0.848375,0.833408,0.501874,0.0,1.0,0.7406
1,0.599149,0.854475,0.804349,0.343203,0.190066,0.702282,0.934367,0.767116,0.221655,0.552314,...,0.157692,0.951564,0.937230,0.918342,0.892617,0.961623,0.472582,0.0,0.0,0.9469
2,0.747348,0.595007,0.480630,1.000000,0.297431,0.245596,0.477305,0.653418,0.477254,0.023106,...,0.383595,0.193926,0.264816,0.511278,0.609447,0.350004,0.330637,1.0,1.0,0.8389
3,0.584317,0.846431,0.752218,0.505116,0.176706,0.710799,0.852595,0.750403,0.281554,0.694501,...,0.210433,0.886611,0.911899,0.913848,0.873774,0.917817,0.459392,0.0,0.0,0.8986
4,0.637460,0.864483,0.793208,0.507068,0.106912,0.677710,0.856693,0.844255,0.255554,0.669354,...,0.185795,0.842202,0.844306,0.866863,0.867919,0.895550,0.602958,1.0,0.0,0.6891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.800321,0.677248,0.735532,0.680884,0.192696,0.774884,0.761011,0.779451,0.292511,0.699359,...,0.147652,0.771311,0.896351,0.768937,0.843793,0.829900,0.414481,1.0,1.0,0.8458
198,0.723619,0.785742,0.652900,0.653066,0.150049,0.757739,0.840444,0.767871,0.302622,0.716947,...,0.122088,0.831093,0.895990,0.870710,0.858337,0.831387,0.498025,1.0,1.0,0.8395
199,0.531755,0.863008,0.746329,0.328289,0.291082,0.643429,0.871264,0.820066,0.219932,0.587813,...,0.158527,0.825005,0.829642,0.836786,0.887809,0.896143,0.443806,0.0,0.0,0.7773
200,0.608258,0.879510,0.848368,0.234444,0.332950,0.762185,0.899370,0.811465,0.235373,0.455455,...,0.244632,0.863275,0.896311,0.901204,0.835798,0.909432,0.370503,0.0,0.0,0.8233


In [13]:
catboost = create_model('catboost')
tunedcat1 = tune_model(catboost, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedcat1)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8519,0.9271,0.8611,0.8158,0.8378,0.7017,0.7025
1,0.8241,0.9127,0.7847,0.8129,0.7986,0.6425,0.6428
2,0.8426,0.9180,0.8056,0.8345,0.8198,0.6801,0.6805
3,0.8241,0.9173,0.8759,0.7651,0.8167,0.6491,0.6546
4,0.8173,0.9098,0.8056,0.7891,0.7973,0.6311,0.6312
Mean,0.8320,0.9170,0.8266,0.8035,0.8140,0.6609,0.6623
Std,0.0130,0.0059,0.0354,0.0240,0.0150,0.0261,0.0259


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.8515,0.9172,0.8068,0.8452,0.8256,0.6964,0.6970


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.670764,0.761575,0.712024,0.517488,0.180527,0.646217,0.823427,0.818446,0.259414,0.692151,...,0.146838,0.755971,0.803129,0.772490,0.848375,0.833408,0.501874,0.0,1.0,0.8321
1,0.599149,0.854475,0.804349,0.343203,0.190066,0.702282,0.934367,0.767116,0.221655,0.552314,...,0.157692,0.951564,0.937230,0.918342,0.892617,0.961623,0.472582,0.0,0.0,0.9621
2,0.747348,0.595007,0.480630,1.000000,0.297431,0.245596,0.477305,0.653418,0.477254,0.023106,...,0.383595,0.193926,0.264816,0.511278,0.609447,0.350004,0.330637,1.0,1.0,0.9571
3,0.584317,0.846431,0.752218,0.505116,0.176706,0.710799,0.852595,0.750403,0.281554,0.694501,...,0.210433,0.886611,0.911899,0.913848,0.873774,0.917817,0.459392,0.0,0.0,0.9079
4,0.637460,0.864483,0.793208,0.507068,0.106912,0.677710,0.856693,0.844255,0.255554,0.669354,...,0.185795,0.842202,0.844306,0.866863,0.867919,0.895550,0.602958,1.0,0.0,0.7400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.800321,0.677248,0.735532,0.680884,0.192696,0.774884,0.761011,0.779451,0.292511,0.699359,...,0.147652,0.771311,0.896351,0.768937,0.843793,0.829900,0.414481,1.0,1.0,0.9189
198,0.723619,0.785742,0.652900,0.653066,0.150049,0.757739,0.840444,0.767871,0.302622,0.716947,...,0.122088,0.831093,0.895990,0.870710,0.858337,0.831387,0.498025,1.0,1.0,0.8934
199,0.531755,0.863008,0.746329,0.328289,0.291082,0.643429,0.871264,0.820066,0.219932,0.587813,...,0.158527,0.825005,0.829642,0.836786,0.887809,0.896143,0.443806,0.0,0.0,0.8884
200,0.608258,0.879510,0.848368,0.234444,0.332950,0.762185,0.899370,0.811465,0.235373,0.455455,...,0.244632,0.863275,0.896311,0.901204,0.835798,0.909432,0.370503,0.0,0.0,0.9025


In [14]:
lightgbm = create_model('lightgbm')
tunedlightgbm = tune_model(lightgbm,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedlightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8488,0.9303,0.8472,0.8188,0.8328,0.6948,0.6951
1,0.8395,0.9169,0.7847,0.8433,0.8129,0.6727,0.6741
2,0.8457,0.9193,0.8333,0.8219,0.8276,0.6879,0.6880
3,0.8272,0.9125,0.8414,0.7871,0.8133,0.6527,0.6540
4,0.8390,0.9107,0.8125,0.8239,0.8182,0.6738,0.6738
Mean,0.8400,0.9179,0.8238,0.8190,0.8210,0.6764,0.6770
Std,0.0074,0.0069,0.0228,0.0181,0.0079,0.0145,0.0141


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8317,0.9126,0.8068,0.8068,0.8068,0.6577,0.6577


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.670764,0.761575,0.712024,0.517488,0.180527,0.646217,0.823427,0.818446,0.259414,0.692151,...,0.146838,0.755971,0.803129,0.772490,0.848375,0.833408,0.501874,0.0,1.0,0.9293
1,0.599149,0.854475,0.804349,0.343203,0.190066,0.702282,0.934367,0.767116,0.221655,0.552314,...,0.157692,0.951564,0.937230,0.918342,0.892617,0.961623,0.472582,0.0,0.0,0.9959
2,0.747348,0.595007,0.480630,1.000000,0.297431,0.245596,0.477305,0.653418,0.477254,0.023106,...,0.383595,0.193926,0.264816,0.511278,0.609447,0.350004,0.330637,1.0,1.0,0.9907
3,0.584317,0.846431,0.752218,0.505116,0.176706,0.710799,0.852595,0.750403,0.281554,0.694501,...,0.210433,0.886611,0.911899,0.913848,0.873774,0.917817,0.459392,0.0,0.0,0.9941
4,0.637460,0.864483,0.793208,0.507068,0.106912,0.677710,0.856693,0.844255,0.255554,0.669354,...,0.185795,0.842202,0.844306,0.866863,0.867919,0.895550,0.602958,1.0,0.0,0.7900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.800321,0.677248,0.735532,0.680884,0.192696,0.774884,0.761011,0.779451,0.292511,0.699359,...,0.147652,0.771311,0.896351,0.768937,0.843793,0.829900,0.414481,1.0,1.0,0.9874
198,0.723619,0.785742,0.652900,0.653066,0.150049,0.757739,0.840444,0.767871,0.302622,0.716947,...,0.122088,0.831093,0.895990,0.870710,0.858337,0.831387,0.498025,1.0,1.0,0.9967
199,0.531755,0.863008,0.746329,0.328289,0.291082,0.643429,0.871264,0.820066,0.219932,0.587813,...,0.158527,0.825005,0.829642,0.836786,0.887809,0.896143,0.443806,0.0,0.0,0.9887
200,0.608258,0.879510,0.848368,0.234444,0.332950,0.762185,0.899370,0.811465,0.235373,0.455455,...,0.244632,0.863275,0.896311,0.901204,0.835798,0.909432,0.370503,0.0,0.0,0.9812


In [22]:
from itertools import combinations
ml = [tunedrf, tunedet, tunedlightgbm, tunedcat1, tunedxgb,tunedmlp]
a2 =  list(combinations(ml, 2))
a3 =  list(combinations(ml, 3))
a4 =  list(combinations(ml, 4))
a5 =  list(combinations(ml, 5))
a6 =  list(combinations(ml, 6))
all = a2 + a3 + a4 + a5 + a6 
print(len(all))
print(len(a2),len(a3),len(a4),len(a5),len(a6))

57
15 20 15 6 1


In [16]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['Accuracy']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8519,0.9276,0.8542,0.8200,0.8367,0.7012,0.7017
1,0.8302,0.9122,0.7778,0.8296,0.8029,0.6541,0.6551
2,0.8426,0.9146,0.7917,0.8444,0.8172,0.6792,0.6803
3,0.8333,0.9097,0.8621,0.7862,0.8224,0.6660,0.6685
4,0.8297,0.9088,0.8194,0.8027,0.8110,0.6561,0.6562
Mean,0.8375,0.9146,0.8210,0.8166,0.8180,0.6713,0.6724
Std,0.0085,0.0068,0.0332,0.0204,0.0114,0.0174,0.0173


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8416,0.9114,0.8182,0.8182,0.8182,0.6778,0.6778


(ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=0, verbose=0,
                     warm_start=False), XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max

In [23]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['AUC']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8519,0.9279,0.8750,0.8077,0.8400,0.7025,0.7044
1,0.8395,0.9144,0.8056,0.8286,0.8169,0.6741,0.6743
2,0.8333,0.9203,0.8194,0.8082,0.8138,0.6630,0.6630
3,0.8333,0.9167,0.8897,0.7725,0.8269,0.6677,0.6739
4,0.8328,0.9148,0.8264,0.8041,0.8151,0.6626,0.6628
Mean,0.8382,0.9188,0.8432,0.8042,0.8225,0.6740,0.6757
Std,0.0073,0.0050,0.0330,0.0181,0.0099,0.0148,0.0152


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8366,0.9150,0.8182,0.8090,0.8136,0.6682,0.6682


(ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='entropy', max_depth=11, max_features=1.0,
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.003114639557041988,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=300, n_jobs=-1, oob_score=False,
                     random_state=0, verbose=0, warm_start=False), LGBMClassifier(bagging_fraction=0.6687470558641753, bagging_freq=3,
               boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.6434460195877356, importance_type='split',
               learning_rate=0.05095675516498231, max_depth=-1,
               min_child_samples=7, min_child_weight=0.001,
               min_split_gain=0.17805250877741555, n_estimators=200, n_jobs=-1,
               num_leaves=225

In [26]:
blend = blend_models([tunedxgb,tunedet,tunedlightgbm,tunedcat1])
md = predict_model(blend,data = test1)
md
y_pred = md['Label']
y_true = md['active']
t = y_pred.astype(float)
t
from sklearn.metrics import balanced_accuracy_score
accuracy = balanced_accuracy_score(y_true, t)
accuracy

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8488,0.9290,0.8681,0.8065,0.8361,0.6961,0.6977
1,0.8333,0.9186,0.7917,0.8261,0.8085,0.6611,0.6616
2,0.8364,0.9198,0.8403,0.8013,0.8203,0.6704,0.6710
3,0.8364,0.9170,0.9034,0.7706,0.8317,0.6745,0.6826
4,0.8328,0.9154,0.8403,0.7961,0.8176,0.6635,0.6643
Mean,0.8376,0.9200,0.8487,0.8001,0.8229,0.6731,0.6754
Std,0.0058,0.0048,0.0368,0.0179,0.0100,0.0124,0.0133


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8966,0.9582,0.9032,0.8750,0.8889,0.7922,0.7925


0.8970674486803518